# 🎤 Common Voice - Ses Analizi Demo Arayüzü

Bu notebook, eğitilmiş modelleri kullanarak:
- 🎙️ **Ses kaydı** veya **dosya yükleme**
- 🤖 **Otomatik tahmin**: Yaş, Cinsiyet, Aksan
- ✅ **Gerçek değerlerle karşılaştırma**
- 📊 **Accuracy ve görselleştirme**

## Gereksinimler
- Eğitilmiş modeller (Google Drive'da)
- Gradio (arayüz için)


In [7]:
# =============================================================================
# GOOGLE DRIVE BAĞLANTISI
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive bağlandı!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive bağlandı!


## 1. Kurulum ve Modelleri Yükle


In [8]:
# =============================================================================
# KURULUM
# =============================================================================

%pip install -q gradio librosa soundfile joblib scikit-learn numpy pandas matplotlib seaborn

print("✅ Kütüphaneler kuruldu!")


✅ Kütüphaneler kuruldu!


In [9]:
# =============================================================================
# KÜTÜPHANELERİ İMPORT ET
# =============================================================================

import numpy as np
import pandas as pd
import librosa
import joblib
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Kütüphaneler yüklendi!")


✅ Kütüphaneler yüklendi!


In [10]:
import os
# Model klasörünü kontrol et
model_path = "/content/drive/MyDrive/elif/big-data/spark-ray"

if os.path.exists(model_path):
    print(f"✅ Klasör mevcut: {model_path}")
    print("\n📁 İçerik:")
    for item in os.listdir(model_path):
        size = os.path.getsize(os.path.join(model_path, item))
        print(f"   📄 {item} ({size/1024:.1f} KB)")
else:
    print(f"❌ Klasör bulunamadı: {model_path}")

✅ Klasör mevcut: /content/drive/MyDrive/elif/big-data/spark-ray

📁 İçerik:
   📄 cv_ray_features (4.0 KB)
   📄 cv_benchmarks (4.0 KB)
   📄 confusion_matrices_ray.png (146.9 KB)
   📄 common_voice_models_ray (4.0 KB)
   📄 spark_vs_ray_comparison.png (84.4 KB)
   📄 1_bireysel_model_accuracy.png (101.2 KB)
   📄 2_kac_etiket_dogru.png (92.9 KB)
   📄 3_tum_kombinasyonlar.png (160.1 KB)
   📄 4_ozet_pasta_grafigi.png (122.3 KB)


In [11]:
# =============================================================================
# MODELLERİ YÜKLE
# =============================================================================

# Model yolu - SENİN DOSYA YAPINA GÖRE
MODEL_PATH = "/content/drive/MyDrive/elif/big-data/spark-ray/common_voice_models_ray"

import os

print(f"📁 Model yolu: {MODEL_PATH}")

# Modelleri yükle
try:
    svm_gender = joblib.load(f"{MODEL_PATH}/svm_gender.pkl")
    svm_age = joblib.load(f"{MODEL_PATH}/svm_age.pkl")
    svm_accent = joblib.load(f"{MODEL_PATH}/svm_accent.pkl")
    scaler = joblib.load(f"{MODEL_PATH}/scaler.pkl")
    le_gender = joblib.load(f"{MODEL_PATH}/le_gender.pkl")
    le_age = joblib.load(f"{MODEL_PATH}/le_age.pkl")
    le_accent = joblib.load(f"{MODEL_PATH}/le_accent.pkl")

    print("\n✅ Tüm modeller başarıyla yüklendi!")
    print(f"\n📋 Sınıflar:")
    print(f"   Cinsiyet: {list(le_gender.classes_)}")
    print(f"   Yaş: {list(le_age.classes_)}")
    print(f"   Aksan: {list(le_accent.classes_)}")
except Exception as e:
    print(f"❌ Model yükleme hatası: {e}")

📁 Model yolu: /content/drive/MyDrive/elif/big-data/spark-ray/common_voice_models_ray

✅ Tüm modeller başarıyla yüklendi!

📋 Sınıflar:
   Cinsiyet: ['female', 'male']
   Yaş: ['middle', 'senior', 'young']
   Aksan: ['african', 'australia', 'canada', 'england', 'indian', 'newzealand', 'scotland', 'us']


In [12]:
# =============================================================================
# MODELLERİ YÜKLE
# =============================================================================

import os

# Model yolu - SENİN DOSYA YAPINA GÖRE
MODEL_PATH = "/content/drive/MyDrive/elif/big-data/spark-ray/common_voice_models_ray"

# Alternatif yolları kontrol et (senin yapına göre)
possible_paths = [
    "/content/drive/MyDrive/elif/big-data/spark-ray/common_voice_models_ray",  # Senin yolun
    "/content/drive/MyDrive/common_voice_models_ray",
    "/content/drive/MyDrive/common_voice_models"
]

for path in possible_paths:
    if os.path.exists(path):
        MODEL_PATH = path
        break

print(f"📁 Model yolu: {MODEL_PATH}")

# Modelleri yükle
try:
    svm_gender = joblib.load(f"{MODEL_PATH}/svm_gender.pkl")
    svm_age = joblib.load(f"{MODEL_PATH}/svm_age.pkl")
    svm_accent = joblib.load(f"{MODEL_PATH}/svm_accent.pkl")
    scaler = joblib.load(f"{MODEL_PATH}/scaler.pkl")
    le_gender = joblib.load(f"{MODEL_PATH}/le_gender.pkl")
    le_age = joblib.load(f"{MODEL_PATH}/le_age.pkl")
    le_accent = joblib.load(f"{MODEL_PATH}/le_accent.pkl")

    print("\n✅ Tüm modeller başarıyla yüklendi!")
    print(f"\n📋 Sınıflar:")
    print(f"   Cinsiyet: {list(le_gender.classes_)}")
    print(f"   Yaş: {list(le_age.classes_)}")
    print(f"   Aksan: {list(le_accent.classes_)}")
except Exception as e:
    print(f"❌ Model yükleme hatası: {e}")
    print("\n💡 Önce CommonVoice_Ray.ipynb notebook'unu çalıştırıp modelleri kaydetmeniz gerekiyor!")

📁 Model yolu: /content/drive/MyDrive/elif/big-data/spark-ray/common_voice_models_ray

✅ Tüm modeller başarıyla yüklendi!

📋 Sınıflar:
   Cinsiyet: ['female', 'male']
   Yaş: ['middle', 'senior', 'young']
   Aksan: ['african', 'australia', 'canada', 'england', 'indian', 'newzealand', 'scotland', 'us']


In [13]:
# =============================================================================
# ÖZELLİK ÇIKARMA FONKSİYONU
# =============================================================================

def extract_features(audio_path, sr=16000, duration=5):
    """
    Ses dosyasından 46 özellik çıkarır

    Özellikler:
    - MFCC (mean + std): 26 boyut
    - Zero Crossing Rate: 2 boyut
    - Spectral Centroid: 2 boyut
    - Spectral Rolloff: 2 boyut
    - Chroma: 12 boyut
    - Spectral Bandwidth: 2 boyut
    TOPLAM: 46 boyut
    """
    try:
        # Ses dosyasını yükle
        y, sr = librosa.load(audio_path, sr=sr, duration=duration)

        # Çok kısa ses için padding
        if len(y) < sr:
            y = np.pad(y, (0, sr - len(y)))

        # 1. MFCC (13 katsayı)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_std = np.std(mfcc, axis=1)

        # 2. Zero Crossing Rate
        zcr = librosa.feature.zero_crossing_rate(y)
        zcr_mean = np.mean(zcr)
        zcr_std = np.std(zcr)

        # 3. Spectral Centroid
        cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        cent_mean = np.mean(cent)
        cent_std = np.std(cent)

        # 4. Spectral Rolloff
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        rolloff_mean = np.mean(rolloff)
        rolloff_std = np.std(rolloff)

        # 5. Chroma (12 frekans bandı)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)

        # 6. Spectral Bandwidth
        bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        bandwidth_mean = np.mean(bandwidth)
        bandwidth_std = np.std(bandwidth)

        # Tüm özellikleri birleştir (46 boyut)
        features = np.concatenate([
            mfcc_mean,                          # 13
            mfcc_std,                           # 13
            [zcr_mean, zcr_std],                # 2
            [cent_mean, cent_std],              # 2
            [rolloff_mean, rolloff_std],        # 2
            chroma_mean,                        # 12
            [bandwidth_mean, bandwidth_std]     # 2
        ])

        return features

    except Exception as e:
        print(f"Hata: {e}")
        return None

print("✅ Özellik çıkarma fonksiyonu hazır!")


✅ Özellik çıkarma fonksiyonu hazır!


In [14]:
# =============================================================================
# TAHMİN FONKSİYONU
# =============================================================================

def predict_audio(audio_path):
    """
    Ses dosyası için Gender, Age, Accent tahmini yapar
    """
    # Özellik çıkar
    features = extract_features(audio_path)

    if features is None:
        return None, None, None

    # Normalize et
    features_scaled = scaler.transform([features])

    # Tahminler
    gender_pred = svm_gender.predict(features_scaled)[0]
    age_pred = svm_age.predict(features_scaled)[0]
    accent_pred = svm_accent.predict(features_scaled)[0]

    # Label'ları decode et
    gender = le_gender.inverse_transform([int(gender_pred)])[0]
    age = le_age.inverse_transform([int(age_pred)])[0]
    accent = le_accent.inverse_transform([int(accent_pred)])[0]

    return gender, age, accent

print("✅ Tahmin fonksiyonu hazır!")


✅ Tahmin fonksiyonu hazır!


In [15]:
# =============================================================================
# TEST GEÇMİŞİ (Session boyunca saklanır)
# =============================================================================

# Test sonuçlarını saklamak için global liste
test_history = []

def add_to_history(pred_gender, pred_age, pred_accent, true_gender, true_age, true_accent):
    """Test sonucunu geçmişe ekle"""
    test_history.append({
        'timestamp': datetime.now().strftime('%H:%M:%S'),
        'pred_gender': pred_gender,
        'pred_age': pred_age,
        'pred_accent': pred_accent,
        'true_gender': true_gender,
        'true_age': true_age,
        'true_accent': true_accent,
        'gender_correct': pred_gender == true_gender,
        'age_correct': pred_age == true_age,
        'accent_correct': pred_accent == true_accent
    })

def calculate_accuracy():
    """Toplam accuracy hesapla"""
    if len(test_history) == 0:
        return 0, 0, 0, 0

    gender_acc = sum(1 for t in test_history if t['gender_correct']) / len(test_history)
    age_acc = sum(1 for t in test_history if t['age_correct']) / len(test_history)
    accent_acc = sum(1 for t in test_history if t['accent_correct']) / len(test_history)
    all_correct = sum(1 for t in test_history if t['gender_correct'] and t['age_correct'] and t['accent_correct']) / len(test_history)

    return gender_acc, age_acc, accent_acc, all_correct

print("✅ Test geçmişi sistemi hazır!")


✅ Test geçmişi sistemi hazır!


## 2. Gradio Arayüzü


In [16]:
# =============================================================================
# GRADIO ARAYÜZ FONKSİYONLARI
# =============================================================================

def process_audio(audio):
    """
    Ses dosyasını işle ve tahmin yap
    """
    if audio is None:
        return "❌ Lütfen bir ses dosyası yükleyin veya kaydedin.", "", "", ""

    # Tahmin yap
    gender, age, accent = predict_audio(audio)

    if gender is None:
        return "❌ Ses işlenemedi. Lütfen tekrar deneyin.", "", "", ""

    # Sonuçları formatla
    result = f"""
🎯 **TAHMİN SONUÇLARI**

👤 **Cinsiyet:** {gender.upper()}
📅 **Yaş Grubu:** {age.upper()}
🗣️ **Aksan:** {accent.upper()}
"""

    return result, gender, age, accent


def compare_results(audio, pred_gender, pred_age, pred_accent, true_gender, true_age, true_accent):
    """
    Tahmin ve gerçek değerleri karşılaştır
    """
    if not pred_gender or not true_gender:
        return "❌ Önce ses yükleyip tahmin yapın, sonra gerçek değerleri seçin.", None

    # Geçmişe ekle
    add_to_history(pred_gender, pred_age, pred_accent, true_gender, true_age, true_accent)

    # Bu test için sonuçlar
    gender_match = "✅" if pred_gender == true_gender else "❌"
    age_match = "✅" if pred_age == true_age else "❌"
    accent_match = "✅" if pred_accent == true_accent else "❌"

    correct_count = sum([pred_gender == true_gender, pred_age == true_age, pred_accent == true_accent])

    # Toplam accuracy
    g_acc, a_acc, ac_acc, all_acc = calculate_accuracy()

    result = f"""
## 📊 KARŞILAŞTIRMA SONUCU

### Bu Test:
| Özellik | Tahmin | Gerçek | Sonuç |
|---------|--------|--------|-------|
| Cinsiyet | {pred_gender} | {true_gender} | {gender_match} |
| Yaş | {pred_age} | {true_age} | {age_match} |
| Aksan | {pred_accent} | {true_accent} | {accent_match} |

**Bu test sonucu:** {correct_count}/3 doğru

---

### 📈 Toplam Performans ({len(test_history)} test):
| Metrik | Doğruluk |
|--------|----------|
| Cinsiyet Accuracy | {g_acc:.1%} |
| Yaş Accuracy | {a_acc:.1%} |
| Aksan Accuracy | {ac_acc:.1%} |
| **Tümü Doğru (Multi-Label)** | **{all_acc:.1%}** |
"""

    # Grafik oluştur
    fig = create_accuracy_chart()

    return result, fig


def create_accuracy_chart():
    """
    Accuracy grafiği oluştur
    """
    if len(test_history) == 0:
        return None

    g_acc, a_acc, ac_acc, all_acc = calculate_accuracy()

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Sol: Bar chart
    ax1 = axes[0]
    categories = ['Cinsiyet', 'Yaş', 'Aksan', 'Tümü Doğru']
    accuracies = [g_acc * 100, a_acc * 100, ac_acc * 100, all_acc * 100]
    colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

    bars = ax1.bar(categories, accuracies, color=colors, edgecolor='black', linewidth=2)
    ax1.set_ylabel('Doğruluk (%)', fontsize=12)
    ax1.set_title(f'📊 Model Performansı ({len(test_history)} test)', fontsize=14, fontweight='bold')
    ax1.set_ylim(0, 110)

    for bar, acc in zip(bars, accuracies):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
                 f'{acc:.1f}%', ha='center', fontsize=12, fontweight='bold')

    # Sağ: Doğru/Yanlış dağılımı
    ax2 = axes[1]

    # Her test için kaç doğru
    correct_counts = []
    for t in test_history:
        count = sum([t['gender_correct'], t['age_correct'], t['accent_correct']])
        correct_counts.append(count)

    count_distribution = [correct_counts.count(i) for i in range(4)]

    labels = ['0 Doğru', '1 Doğru', '2 Doğru', '3 Doğru']
    colors_pie = ['#e74c3c', '#f39c12', '#3498db', '#27ae60']

    if sum(count_distribution) > 0:
        wedges, texts, autotexts = ax2.pie(count_distribution, labels=labels, colors=colors_pie,
                                            autopct=lambda pct: f'{pct:.1f}%' if pct > 0 else '',
                                            startangle=90, textprops={'fontsize': 10})
        ax2.set_title('📊 Doğru Tahmin Dağılımı', fontsize=14, fontweight='bold')

    plt.tight_layout()

    return fig


def reset_history():
    """Test geçmişini sıfırla"""
    global test_history
    test_history = []
    return "✅ Test geçmişi sıfırlandı!", None


def get_history_table():
    """Test geçmişini tablo olarak göster"""
    if len(test_history) == 0:
        return pd.DataFrame()

    df = pd.DataFrame(test_history)
    df['all_correct'] = df['gender_correct'] & df['age_correct'] & df['accent_correct']

    # Emoji ekle
    df['gender_result'] = df['gender_correct'].apply(lambda x: '✅' if x else '❌')
    df['age_result'] = df['age_correct'].apply(lambda x: '✅' if x else '❌')
    df['accent_result'] = df['accent_correct'].apply(lambda x: '✅' if x else '❌')

    display_df = df[['timestamp', 'pred_gender', 'true_gender', 'gender_result',
                     'pred_age', 'true_age', 'age_result',
                     'pred_accent', 'true_accent', 'accent_result']]

    display_df.columns = ['Zaman', 'Tahmin (C)', 'Gerçek (C)', 'C',
                          'Tahmin (Y)', 'Gerçek (Y)', 'Y',
                          'Tahmin (A)', 'Gerçek (A)', 'A']

    return display_df

print("✅ Arayüz fonksiyonları hazır!")


✅ Arayüz fonksiyonları hazır!


In [17]:
# =============================================================================
# GRADIO ARAYÜZÜ OLUŞTUR
# =============================================================================

# Seçenekler
gender_options = list(le_gender.classes_)
age_options = list(le_age.classes_)
accent_options = list(le_accent.classes_)

# CSS stili
custom_css = """
.gradio-container {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
.gr-button-primary {
    background-color: #3498db !important;
}
"""

# Arayüz
with gr.Blocks(title="🎤 Common Voice Demo", css=custom_css, theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
    # 🎤 Common Voice - Ses Analizi Demo

    Bu demo, eğitilmiş ML modelleri kullanarak seslerden **cinsiyet**, **yaş** ve **aksan** tahmini yapar.

    ### Nasıl Kullanılır:
    1. 🎙️ **Ses Kaydet** veya **Dosya Yükle**
    2. 🤖 **"Tahmin Yap"** butonuna tıkla
    3. ✅ Gerçek değerleri seç ve **"Karşılaştır"** butonuna tıkla
    4. 📊 Sonuçları ve performansı gör!
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🎙️ Ses Girişi")
            audio_input = gr.Audio(
                sources=["microphone", "upload"],
                type="filepath",
                label="Ses Kaydı veya Dosya"
            )
            predict_btn = gr.Button("🤖 Tahmin Yap", variant="primary", size="lg")

            prediction_output = gr.Markdown(label="Tahmin Sonuçları")

            # Gizli tahmin değerleri
            pred_gender = gr.Textbox(visible=False)
            pred_age = gr.Textbox(visible=False)
            pred_accent = gr.Textbox(visible=False)

        with gr.Column(scale=1):
            gr.Markdown("### ✅ Gerçek Değerler (Doğrulama İçin)")
            true_gender = gr.Dropdown(
                choices=gender_options,
                label="Gerçek Cinsiyet",
                info="Konuşmacının gerçek cinsiyeti"
            )
            true_age = gr.Dropdown(
                choices=age_options,
                label="Gerçek Yaş Grubu",
                info="young (teens-30s), middle (40s-50s), senior (60+)"
            )
            true_accent = gr.Dropdown(
                choices=accent_options,
                label="Gerçek Aksan",
                info="Konuşmacının aksanı"
            )

            compare_btn = gr.Button("📊 Karşılaştır", variant="secondary", size="lg")

    gr.Markdown("---")
    gr.Markdown("### 📈 Sonuçlar")

    with gr.Row():
        comparison_output = gr.Markdown(label="Karşılaştırma Sonuçları")
        accuracy_chart = gr.Plot(label="Performans Grafiği")

    gr.Markdown("---")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📋 Test Geçmişi")
            history_table = gr.Dataframe(label="Tüm Testler")

            with gr.Row():
                refresh_btn = gr.Button("🔄 Geçmişi Yenile")
                reset_btn = gr.Button("🗑️ Geçmişi Sıfırla", variant="stop")

    # Event handlers
    predict_btn.click(
        fn=process_audio,
        inputs=[audio_input],
        outputs=[prediction_output, pred_gender, pred_age, pred_accent]
    )

    compare_btn.click(
        fn=compare_results,
        inputs=[audio_input, pred_gender, pred_age, pred_accent, true_gender, true_age, true_accent],
        outputs=[comparison_output, accuracy_chart]
    )

    refresh_btn.click(
        fn=get_history_table,
        outputs=[history_table]
    )

    reset_btn.click(
        fn=reset_history,
        outputs=[comparison_output, accuracy_chart]
    )

    gr.Markdown("""
    ---
    ### 📝 Notlar:
    - **Cinsiyet Sınıfları:** female, male
    - **Yaş Sınıfları:** young (teens-30s), middle (40s-50s), senior (60+)
    - **Aksan Sınıfları:** african, australia, canada, england, indian, newzealand, scotland, us
    - Model **SVM RBF** algoritması ile eğitildi
    - Ses dosyası en az **1 saniye** olmalıdır
    """)

print("✅ Gradio arayüzü oluşturuldu!")
print("\n🚀 Arayüzü başlatmak için aşağıdaki hücreyi çalıştırın.")


✅ Gradio arayüzü oluşturuldu!

🚀 Arayüzü başlatmak için aşağıdaki hücreyi çalıştırın.


In [18]:
# =============================================================================
# ARAYÜZÜ BAŞLAT
# =============================================================================

# Colab'da public link ile paylaş
demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8ee97094d0b6fb3d48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8ee97094d0b6fb3d48.gradio.live


In [21]:
# Colab'da çalıştır:
import shutil
from google.colab import files

MODEL_PATH = "/content/drive/MyDrive/elif/big-data/spark-ray/common_voice_models_ray"
shutil.make_archive("/content/models", 'zip', MODEL_PATH)
files.download("/content/models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>